In [2]:
import requests
import pandas as pd
from datetime import datetime, timedelta

In [3]:
import pandas as pd
import requests

def get_crypto_data(crypto_id, days=365):
    url = f"https://api.coingecko.com/api/v3/coins/{crypto_id}/market_chart"
    params = {
        'vs_currency': 'usd',
        'days': days,
        'interval': 'daily'
    }
    
    headers = {'User-Agent': 'Mozilla/5.0'}
    response = requests.get(url, params=params, headers=headers)
    data = response.json()

    if 'prices' not in data:
        print(f"No 'prices' found for {crypto_id}")
        return pd.DataFrame()

    df = pd.DataFrame(data['prices'], columns=['timestamp', 'price'])
    df['date'] = pd.to_datetime(df['timestamp'], unit='ms')

    df['total_volume'] = [x[1] for x in data['total_volumes']]
    df['market_cap'] = [x[1] for x in data['market_caps']]
    df['coin_name'] = crypto_id

    # ➡️ Now select only required columns
    df = df[['date', 'price', 'total_volume', 'market_cap', 'coin_name']]
    
    return df


In [4]:
btc_data = get_crypto_data('bitcoin')

In [5]:
btc_data

,date,price,total_volume,market_cap,coin_name
0,2024-04-28 00:00:00,63517.248743,1.919048e+10,1.249495e+12,bitcoin
1,2024-04-29 00:00:00,63030.433617,1.618029e+10,1.240725e+12,bitcoin
2,2024-04-30 00:00:00,63797.679636,2.724298e+10,1.257159e+12,bitcoin
3,2024-05-01 00:00:00,60749.472092,3.953311e+10,1.197774e+12,bitcoin
4,2024-05-02 00:00:00,58297.574075,5.110938e+10,1.151677e+12,bitcoin
...,...,...,...,...,...
361,2025-04-24 00:00:00,93605.452309,3.990152e+10,1.858660e+12,bitcoin
362,2025-04-25 00:00:00,93872.814229,3.003125e+10,1.864298e+12,bitcoin
363,2025-04-26 00:00:00,94773.112006,3.933618e+10,1.882122e+12,bitcoin
364,2025-04-27 00:00:00,94644.066371,1.544539e+10,1.879132e+12,bitcoin


In [6]:
btc_data.head()

,date,price,total_volume,market_cap,coin_name
0,2024-04-28,63517.248743,1.919048e+10,1.249495e+12,bitcoin
1,2024-04-29,63030.433617,1.618029e+10,1.240725e+12,bitcoin
2,2024-04-30,63797.679636,2.724298e+10,1.257159e+12,bitcoin
3,2024-05-01,60749.472092,3.953311e+10,1.197774e+12,bitcoin
4,2024-05-02,58297.574075,5.110938e+10,1.151677e+12,bitcoin


In [7]:
btc_data.to_csv('crypto_market_data.csv', index=False)

In [8]:
import pandas as pd

# Read both CSV files
df1 = pd.read_csv('C:/Users/somya/Downloads/Hybrid Model for Crypto Volatility Prediction using Sentiment Analysis/Data_set/bitcoin.csv')
df2 = pd.read_csv('C:/Users/somya/Downloads/Hybrid Model for Crypto Volatility Prediction using Sentiment Analysis/Data_set/crypto_market_data.csv')


# Merge them vertically (i.e., one after another)
merged_df = pd.concat([df1, df2], axis=0, ignore_index=True)

# Save the merged file
merged_df.to_csv('C:/Users/somya/Downloads/Hybrid Model for Crypto Volatility Prediction using Sentiment Analysis/Data_set/bitcon_merged_file.csv', index=False)

print("CSV files merged successfully!")


CSV files merged successfully!


In [9]:
def calculate_volatility(merged_df, crypto_id, window=30):
    # Calculate daily returns
    merged_df[f'{crypto_id}_returns'] = merged_df['price'].pct_change()

    # Calculate rolling volatility over 'window' days and annualize it
    merged_df[f'{crypto_id}_volatility'] = merged_df[f'{crypto_id}_returns'].rolling(window=window).std() * (365**0.5)
    
    return merged_df


In [10]:
btc_data = calculate_volatility(merged_df,'bitcoin')

In [11]:
btc_data.isnull().sum()

date                   0
price                  0
total_volume           0
market_cap             1
coin_name              0
bitcoin_returns        1
bitcoin_volatility    30
dtype: int64

In [12]:
btc_data['bitcoin_returns']=btc_data['bitcoin_returns'].fillna(btc_data['bitcoin_returns'].mean())

In [13]:
btc_data['bitcoin_volatility']=btc_data['bitcoin_volatility'].fillna(btc_data['bitcoin_volatility'].mean())

In [14]:
btc_data.tail()

,date,price,total_volume,market_cap,coin_name,bitcoin_returns,bitcoin_volatility
3734,2025-04-24 00:00:00,93605.452309,3.990152e+10,1.858660e+12,bitcoin,0.000313,0.579362
3735,2025-04-25 00:00:00,93872.814229,3.003125e+10,1.864298e+12,bitcoin,0.002856,0.579358
3736,2025-04-26 00:00:00,94773.112006,3.933618e+10,1.882122e+12,bitcoin,0.009591,0.578856
3737,2025-04-27 00:00:00,94644.066371,1.544539e+10,1.879132e+12,bitcoin,-0.001362,0.579085
3738,2025-04-27 20:32:57,94320.952422,1.587970e+10,1.872924e+12,bitcoin,-0.003414,0.564953


In [16]:

btc_data.to_csv('C:/Users/somya/Downloads/Hybrid Model for Crypto Volatility Prediction using Sentiment Analysis/Data_set/bitcon_merged_volatility_file.csv', index=False)